# Label classifier (dSprites): data collection

**Author**: Maleakhi A. Wijaya  
**Description**: This notebook contains code used to collect experimentation data. We compare the performance of methods discussed in Rabanset et al. against our proposed CBSD method.

In [1]:
# Load utilities functions
%run ../../scripts/dsprites_utils.py
%run ../../scripts/shift_applicator.py
%run ../../scripts/shift_dimensionality_reductor.py
%run ../../scripts/constants.py
%run ../../scripts/experiment_utils.py
%run ../../scripts/shift_statistical_test.py

## Load dataset

In [6]:
SEED = 20
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [7]:
path = "../../data/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz"
X_train, X_test, y_train, y_test, c_train, c_test = load_dsprites(path, 100000, train_size=0.85, class_index=1)
n_classes_shape = 3

# Split training into validation set as well 
X_train, X_valid = X_train[:70000], X_train[70000:]
y_train, y_valid = y_train[:70000], y_train[70000:]
c_train, c_valid = c_train[:70000], c_train[70000:]

Training samples: 85000
Testing samples: 15000


In [8]:
# Reshape to appropriate shift input
# It is noteworthy that for efficiency, we represent the images as only 2 dimension
# when we preprocessing (number of instances/ batch size * flatten size).
# When visualising back the image, we need to reshape it back to the original dimension
ORIGINAL_SHAPE = X_test.shape[1:] # constant hold the image original shape
X_test_flatten = deepcopy(X_test.reshape(X_test.shape[0], -1))
X_train_flatten = deepcopy(X_train.reshape(X_train.shape[0], -1))
X_valid_flatten = deepcopy(X_valid.reshape(X_valid.shape[0], -1))

## Dimensionality reduction

We implemented various dimensionality reduction methods, amounting to:
- End to end model (label classifiers/ BBSD)
- Concept bottleneck model (CBSD)
- Trained and untrained autoencoders (TAE and UAE)
- Principal component analysis (PCA)
- Sparse random projection (SRP)

### End-to-end model

In [9]:
path = "../../models/end_to_end_dsprites"
# For training and saving
# histories, end_to_end_model = end_to_end_neural_network(n_classes_shape, Dataset.DSPRITES, 
#                          X_train, y_train, X_valid, y_valid,
#                          path)

# For loading
end_to_end_model = tf.keras.models.load_model(path)

In [58]:
# Evaluate model
y_pred = end_to_end_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4935
           1       1.00      1.00      1.00      5051
           2       1.00      1.00      1.00      5014

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000



### Concept bottleneck model